<a href="https://colab.research.google.com/github/Bagusalfarizi/KI/blob/main/Task%5CTaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Nama :Bagus Al Farizi** \\
 **NPM :2206048700** \\

 <font color="#4863A0">**Kelas Komputasi Intelegensia**


 source : https://huggingface.co/Aardiiiiy/indobertweet-base-Indonesian-sentiment-analysis#model-details

In [24]:
# Install necessary libraries
!pip install transformers torch

In [32]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Aardiiiiy/indobertweet-base-Indonesian-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("Aardiiiiy/indobertweet-base-Indonesian-sentiment-analysis")
sentiment_labels = {0: "Negative", 1: "Neutral", 2: "Positive"}

In [70]:
def predict_sentiment(text, max_length=128):
    # Tokenize the input text with a defined max_length
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_length)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=1)

    # Get the predicted sentiment and score
    sentiment = torch.argmax(predictions, dim=1).item()
    score = predictions[0][sentiment].item()

    # Define sentiment labels
    sentiment_labels = {0: "Negative", 1: "Neutral", 2: "Positive"}

    return sentiment_labels[sentiment], score

# Example usage
text = "warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !"
sentiment, score = predict_sentiment(text)
print(f"Sentiment: {sentiment}, Score: {score:.4f}")

Sentiment: Positive, Score: 0.9793


In [39]:
from sklearn.metrics import accuracy_score

def evaluate_model(texts, labels, max_length=128):
    predictions = []

    print(f"{'Text':<50} {'Predicted Label':<15} {'Ground Truth':<15}")
    print("-" * 80)

    for text, true_label in zip(texts, labels):
        sentiment, _ = predict_sentiment(text, max_length)
        predicted_label = list(sentiment_labels.values()).index(sentiment)  # Map sentiment to corresponding label
        predictions.append(predicted_label)

        # Print the text, predicted label, and ground truth
        print(f"{text:<50} {predicted_label:<15} {true_label:<15}")

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    return accuracy

texts = [
    "Saya sangat senang!",
    "Ini sangat buruk.",
    "Biasa saja.",
    "Dimana dia selama ini",
    "Mengapa produk ini tidak berguna",
    "Layanan pelanggan sangat membantu!",
    "Pengiriman sangat lambat dan mengecewakan.",
    "Film ini sangat membosankan.",
    "Makanannya luar biasa lezat.",
    "Aplikasi ini sering mengalami bug.",
    "Pemandangan di tempat itu sangat indah.",
    "Saya tidak akan merekomendasikan produk ini.",
    "Fasilitas hotelnya sangat bagus.",
    "Ponsel ini terlalu mahal untuk kualitasnya.",
    "Pertunjukannya sangat menarik!"
]

labels = [2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2]  # Ground truth labels (Positive=2, Negative=0, Neutral=1)
accuracy = evaluate_model(texts, labels)
print(f"Accuracy: {accuracy:.4f}")


Text                                               Predicted Label Ground Truth   
--------------------------------------------------------------------------------
Saya sangat senang!                                2               2              
Ini sangat buruk.                                  0               0              
Biasa saja.                                        1               1              
Dimana dia selama ini                              0               0              
Mengapa produk ini tidak berguna                   0               0              
Layanan pelanggan sangat membantu!                 2               2              
Pengiriman sangat lambat dan mengecewakan.         0               0              
Film ini sangat membosankan.                       0               0              
Makanannya luar biasa lezat.                       2               2              
Aplikasi ini sering mengalami bug.                 0               0              
Pemand

In [44]:
!pip install datasets

In [69]:
from datasets import load_dataset

# Load the IndoNLU SMSA dataset
dataset = load_dataset('indonlu', 'smsa')

# Function to display examples from a dataset
def display_examples(data, num_examples=5):
    for i in range(num_examples):
        print(f"Example {i+1}:")
        print(f"Text: {data[i]['text']}")
        print(f"Label: {data[i]['label']}")
        print("-" * 40)

# Display examples from the training set
print("Training Set Examples:")
display_examples(dataset['train'], num_examples=5)

# Display examples from the test set
print("\nTest Set Examples:")
display_examples(dataset['test'], num_examples=5)


Training Set Examples:
Example 1:
Text: warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !
Label: 0
----------------------------------------
Example 2:
Text: mohon ulama lurus dan k212 mmbri hujjah partai apa yang harus diwlh agar suara islam tidak pecah-pecah
Label: 1
----------------------------------------
Example 3:
Text: lokasi strategis di jalan sumatera bandung . tempat nya nyaman terutama sofa di lantai 2 . paella nya enak , sangat pas dimakan dengan minum bir dingin . appetiser nya juga enak-enak .
Label: 0
----------------------------------------
Example 4:
Text: betapa bahagia nya diri ini saat unboxing paket dan barang nya bag

In [64]:
# Function to predict sentiment for a given text
def predict_sentiment(text, max_length=128):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_length)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=-1).item()

    # Get the sentiment label string based on the predicted label
    sentiment_label = list(sentiment_labels.keys())[predicted_label]  # Assuming sentiment_labels is a dictionary

    # Return the sentiment label string and the predicted numeric label
    return sentiment_label, predicted_label

# Evaluation function
def evaluate_model_dataset(texts, labels, max_length=128):
    predictions = []

    # Loop through the texts and make predictions
    for text in texts:
        sentiment, predicted_label = predict_sentiment(text, max_length)  # Unpack correctly
        #predicted_label = list(sentiment_labels.values()).index(sentiment)  # No longer needed, using predicted_label directly
        predictions.append(predicted_label)

    # Calculate accuracy using ground truth labels and predictions
    accuracy = accuracy_score(labels, predictions)

    return accuracy

# Load an appropriate dataset from IndoNLU (e.g., 'smsa' or other)
dataset = load_dataset('indonlu', 'smsa')

# Extract test data
test_dataset = dataset['test']
texts = test_dataset['text']
labels = test_dataset['label']

# Evaluate model on test set of IndoNLU SMSA
accuracy = evaluate_model_dataset(texts, labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.1600


In [71]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class TransformerWithAttention(nn.Module):
    def __init__(self, pretrained_model_name):
        super(TransformerWithAttention, self).__init__()
        # Load pre-trained model and tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
        self.model = AutoModel.from_pretrained(pretrained_model_name)

        # Define additional layers if necessary
        self.fc = nn.Linear(self.model.config.hidden_size, 3)  # Assuming 3 classes for sentiment

    def forward(self, input_text):
        # Tokenize and prepare inputs
        inputs = self.tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

        # Get the outputs from the pre-trained model
        outputs = self.model(**inputs)

        # Use the hidden states from the last layer (or pooled output)
        hidden_states = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)

        # Apply attention mechanism (mean pooling in this case)
        attention_output = torch.mean(hidden_states, dim=1)  # Shape: (batch_size, hidden_size)

        # Final classification layer
        logits = self.fc(attention_output)
        return logits

# Example usage
if __name__ == "__main__":
    # Create an instance of the Transformer class
    model_name = "Aardiiiiy/indobertweet-base-Indonesian-sentiment-analysis"
    model = TransformerWithAttention(model_name)

    # Sample input text for testing
    sample_text = ["Saya sangat senang dengan produk ini!", "Ini sangat buruk."]

    # Forward pass
    with torch.no_grad():
        output = model(sample_text)

    print(output)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([[-0.3119,  0.2876, -0.4163],
        [ 0.0868,  0.0149,  0.0211]])


In [73]:
import torch
import torch.nn.functional as F

logits = torch.tensor([[-0.3119,  0.2876, -0.4163],
                        [ 0.0868,  0.0149,  0.0211]])

# Apply softmax to get probabilities
probabilities = F.softmax(logits, dim=1)
print(probabilities)

tensor([[0.2687, 0.4893, 0.2420],
        [0.3488, 0.3246, 0.3266]])


In [75]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics import accuracy_score

# Define the Transformer model with attention
class TransformerWithAttention(nn.Module):
    def __init__(self, pretrained_model_name):
        super(TransformerWithAttention, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
        self.model = AutoModel.from_pretrained(pretrained_model_name)
        self.fc = nn.Linear(self.model.config.hidden_size, 3)  # 3 classes for sentiment

    def forward(self, input_text):
        inputs = self.tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
        outputs = self.model(**inputs)
        hidden_states = outputs.last_hidden_state
        attention_output = torch.mean(hidden_states, dim=1)
        logits = self.fc(attention_output)
        return logits

def evaluate_model(model, texts, labels):
    model.eval()  # Set the model to evaluation mode
    predictions = []

    with torch.no_grad():  # Disable gradient calculation
        for text in texts:
            output = model(text)  # Forward pass
            probabilities = F.softmax(output, dim=1)  # Get probabilities
            predicted_class = torch.argmax(probabilities, dim=1).item()  # Get the predicted class
            predictions.append(predicted_class)

    # Print predictions vs actual labels
    for text, pred, label in zip(texts, predictions, labels):
        print(f"Text: {text} | Predicted: {pred} | Actual: {label}")

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)
    return accuracy


# Synthetic dataset
texts = [
    "Saya sangat senang!",
    "Ini sangat buruk.",
    "Biasa saja.",
    "Dimana dia selama ini",
    "Mengapa produk ini tidak berguna",
    "Layanan pelanggan sangat membantu!",
    "Pengiriman sangat lambat dan mengecewakan.",
    "Film ini sangat membosankan.",
    "Makanannya luar biasa lezat.",
    "Aplikasi ini sering mengalami bug.",
    "Pemandangan di tempat itu sangat indah.",
    "Saya tidak akan merekomendasikan produk ini.",
    "Fasilitas hotelnya sangat bagus.",
    "Ponsel ini terlalu mahal untuk kualitasnya.",
    "Pertunjukannya sangat menarik!"
]

labels = [2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2]  # Ground truth labels

# Load the model and evaluate
model_name = "Aardiiiiy/indobertweet-base-Indonesian-sentiment-analysis"
model = TransformerWithAttention(model_name)

# Evaluate the model on the synthetic dataset
accuracy = evaluate_model(model, texts, labels)
print(f"Model Accuracy: {accuracy:.4f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text: Saya sangat senang! | Predicted: 1 | Actual: 2
Text: Ini sangat buruk. | Predicted: 2 | Actual: 0
Text: Biasa saja. | Predicted: 0 | Actual: 1
Text: Dimana dia selama ini | Predicted: 1 | Actual: 0
Text: Mengapa produk ini tidak berguna | Predicted: 2 | Actual: 0
Text: Layanan pelanggan sangat membantu! | Predicted: 1 | Actual: 2
Text: Pengiriman sangat lambat dan mengecewakan. | Predicted: 1 | Actual: 0
Text: Film ini sangat membosankan. | Predicted: 2 | Actual: 0
Text: Makanannya luar biasa lezat. | Predicted: 1 | Actual: 2
Text: Aplikasi ini sering mengalami bug. | Predicted: 1 | Actual: 0
Text: Pemandangan di tempat itu sangat indah. | Predicted: 0 | Actual: 2
Text: Saya tidak akan merekomendasikan produk ini. | Predicted: 0 | Actual: 0
Text: Fasilitas hotelnya sangat bagus. | Predicted: 1 | Actual: 2
Text: Ponsel ini terlalu mahal untuk kualitasnya. | Predicted: 2 | Actual: 0
Text: Pertunjukannya sangat menarik! | Predicted: 1 | Actual: 2
Model Accuracy: 0.0667


In [76]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import accuracy_score

# Define the Transformer model with attention
class TransformerWithAttention(nn.Module):
    def __init__(self, pretrained_model_name):
        super(TransformerWithAttention, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
        self.model = AutoModel.from_pretrained(pretrained_model_name)
        self.fc = nn.Linear(self.model.config.hidden_size, 3)  # 3 classes for sentiment

    def forward(self, input_text):
        inputs = self.tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
        outputs = self.model(**inputs)
        hidden_states = outputs.last_hidden_state
        attention_output = torch.mean(hidden_states, dim=1)
        logits = self.fc(attention_output)
        return logits

# Evaluation function
def evaluate_model(model, texts, labels):
    model.eval()  # Set the model to evaluation mode
    predictions = []

    with torch.no_grad():  # Disable gradient calculation
        for text in texts:
            output = model(text)  # Forward pass
            probabilities = F.softmax(output, dim=1)  # Get probabilities
            predicted_class = torch.argmax(probabilities, dim=1).item()  # Get the predicted class
            predictions.append(predicted_class)

    # Print predictions vs actual labels
    for text, pred, label in zip(texts, predictions, labels):
        print(f"Text: {text} | Predicted: {pred} | Actual: {label}")

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)
    return accuracy

# Load the IndoNLU SMSA dataset
dataset = load_dataset('indonlu', 'smsa')

# Use dataset['test'] from IndoNLU SMSA
test_dataset = dataset['test']

# Extract texts and labels from the test dataset
texts = test_dataset['text']
labels = test_dataset['label']

# Load the model and evaluate
model_name = "Aardiiiiy/indobertweet-base-Indonesian-sentiment-analysis"
model = TransformerWithAttention(model_name)

# Evaluate the model on the IndoNLU SMSA test set
accuracy = evaluate_model(model, texts, labels)
print(f"Model Accuracy: {accuracy:.4f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text: kemarin gue datang ke tempat makan baru yang ada di dago atas . gue kira makanan nya enak karena harga nya mahal . ternyata , boro-boro . tidak mau lagi deh ke tempat itu . sudah mana tempat nya juga tidak nyaman banget , terlalu sempit . | Predicted: 2 | Actual: 2
Text: kayak nya sih gue tidak akan mau balik lagi ke tempat itu . gila , ya , gue enggak ngerti kenapa tempat nya dibiarkan panas . sudah begitu kotor pula . kalau panas kepanasan , kalau hujan kehujanan . harus nya sih tidak ada restoran yang kayak gitu . tidak tahu deh apa yang mereka jual . | Predicted: 1 | Actual: 2
Text: kalau dipikir-pikir , sebenarnya tidak ada yang bisa dibanggakan dari jokowi . pertama , dia tidak bisa nepatin janji . kedua , kerjaan nya selalu pencitraan . ketiga , dia tidak pro rakyat . sudahlah . ku sudah terlanjur kecewa . | Predicted: 1 | Actual: 2
Text: ini pertama kalinya gua ke bank buat ngurusin pembuatan rekening baru . nama nya juga orang pertama kali ya baru ke bank , gua kena semp